# Humor Classifier Baseline
- Inspired by the following resources:
    - https://huggingface.co/docs/transformers/training
    - https://huggingface.co/docs/transformers/tasks/sequence_classification
    - https://medium.com/mlearning-ai/fine-tuning-bert-for-tweets-classification-ft-hugging-face-8afebadd5dbf

In [1]:
!pip install datasets --quiet
!pip install transformers --quiet
!pip install evaluate --quiet
!pip install torch --quiet
!pip install wandb --quiet
!pip install scikit-learn --quiet

In [2]:
from datasets import load_dataset

from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import get_scheduler

import evaluate

import torch
from torch.utils.data import DataLoader
from torch.optim import AdamW

import wandb

from tqdm.auto import tqdm

import json

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Load Data

In [3]:
dataset = load_dataset('csv', data_files={'train': './data/oasst1_train_cleaned.csv', 'test': './data/oasst1_test_cleaned.csv'})

Found cached dataset csv (/home/jovyan/.cache/huggingface/datasets/csv/default-65c47b1d86afb971/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
100%|██████████| 2/2 [00:00<00:00, 571.86it/s]


### Prepare Data

In [4]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

In [5]:
def transform_labels(data):
    humor = data['humor']
    label = 1 if humor > 0 else 0
    return {'labels': label}

dataset = dataset.map(transform_labels)

Loading cached processed dataset at /home/jovyan/.cache/huggingface/datasets/csv/default-65c47b1d86afb971/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-46f084f3920b7be4.arrow
Loading cached processed dataset at /home/jovyan/.cache/huggingface/datasets/csv/default-65c47b1d86afb971/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-41f22b9ad45430c4.arrow


In [6]:
def tokenize_data(data):
    return tokenizer(data['text'], padding="max_length", truncation=True)

remove_columns = ['message_id', 'role', 'lang', 'humor', 'text']
dataset = dataset.map(tokenize_data, remove_columns=remove_columns, batched=True)

Loading cached processed dataset at /home/jovyan/.cache/huggingface/datasets/csv/default-65c47b1d86afb971/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-7340d8cd191102e3.arrow


In [7]:
dataset.set_format(type="torch", columns=["input_ids", "token_type_ids", "attention_mask", "labels"])

### Training- and Validationsplit

In [8]:
train_validation_dataset = dataset['train']
test_dataset = dataset['test']

In [9]:
train_validation_dataset = train_validation_dataset.train_test_split(test_size=0.1)
train_dataset = train_validation_dataset['train']
validation_dataset = train_validation_dataset['test']

### Data Loader

In [10]:
eval_dataloader = DataLoader(validation_dataset, batch_size=8)

### Setup Model

In [11]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [12]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

### Evaluate

In [13]:
metric = evaluate.load("accuracy")
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

{'accuracy': 0.363395225464191}